In [ ]:
# 25.11.24 13.36 на данный момент актуальный способ расчета косинусного сходства. 
# сохранил в гигантский датасет df_cos1 = pd.read_csv('../vkrecsys/cos_sim_output1.csv') 5,3Гб
# считает батчами по 1000 штук. хз что по точнисти, но думаю, не принципиально, может лучше к датасету и не обращаться.

In [1]:
import pandas as pd
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Чтение данных из parquet файла
df = pd.read_parquet('../vkrecsys/items_meta.parquet.parquet')

# Извлечение массива эмбеддингов
embeddings = np.stack(df['embeddings'])

# Размер одной партии
batch_size = 1000

# Создаем пустой список для хранения результатов
cosine_similarities = []

# Обрабатываем данные партиями
for i in range(0, len(embeddings), batch_size):
    batch_embeddings = embeddings[i:i + batch_size]
    
    # Применяем SVD для уменьшения размерности
    U, s, Vh = svds(batch_embeddings, k=30)  # выбираем количество компонент (k=5)
    reduced_batch_embeddings = U @ np.diag(s)
    
    # Вычисляем косинусную схожесть внутри партии
    batch_cosine_sims = cosine_similarity(reduced_batch_embeddings)
    
    # Добавляем результаты текущей партии в общий список
    cosine_similarities.extend(batch_cosine_sims)

# Добавим результаты обратно в DataFrame
df['cosine_similarities'] = cosine_similarities


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337727 entries, 0 to 337726
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   item_id              337727 non-null  uint32
 1   source_id            337727 non-null  uint32
 2   duration             337727 non-null  uint8 
 3   embeddings           337727 non-null  object
 4   cosine_similarities  337727 non-null  object
dtypes: object(2), uint32(2), uint8(1)
memory usage: 8.1+ MB


In [2]:
df.head()

,item_id,source_id,duration,embeddings,cosine_similarities
0,0,1869,13,"[0.07607838, 0.24312432, -0.07126159, 0.068710...","[0.99999994, 0.1686182, 0.36216357, 0.30833608..."
1,1,1869,8,"[-0.2897247, 0.016220074, -0.14935663, 0.08754...","[0.1686182, 1.0000001, 0.62054056, 0.5462112, ..."
2,2,1869,10,"[-0.08479501, 0.16359554, -0.12039807, 0.09242...","[0.36216357, 0.62054056, 0.99999994, 0.6539094..."
3,3,1869,49,"[-0.061852332, 0.14139329, -0.00040801117, -0....","[0.30833608, 0.5462112, 0.6539094, 1.0000001, ..."
4,4,1869,8,"[-0.12159933, -0.019279012, -0.13987888, 0.020...","[0.38104185, 0.74441653, 0.84191394, 0.6869687..."


In [8]:
# Сохранить результат в новый файл
df.to_csv('cos_sim_output1.csv', index=False)

In [9]:
df_cos1 = pd.read_csv('../vkrecsys/cos_sim_output1.csv')

In [10]:
df_cos1.head()

,item_id,source_id,duration,embeddings,cosine_similarities
0,0,1869,13,[ 7.60783777e-02 2.43124321e-01 -7.12615922e-...,[ 9.99999940e-01 1.68618202e-01 3.62163574e-...
1,1,1869,8,[-0.2897247 0.01622007 -0.14935663 0.087541...,[ 1.68618202e-01 1.00000012e+00 6.20540559e-...
2,2,1869,10,[-0.08479501 0.16359554 -0.12039807 0.092425...,[ 3.62163574e-01 6.20540559e-01 9.99999940e-...
3,3,1869,49,[-0.06185233 0.14139329 -0.00040801 -0.024462...,[ 3.08336079e-01 5.46211183e-01 6.53909385e-...
4,4,1869,8,[-0.12159933 -0.01927901 -0.13987888 0.020500...,[ 3.81041855e-01 7.44416535e-01 8.41913939e-...


In [11]:
df_cos1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337727 entries, 0 to 337726
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   item_id              337727 non-null  int64 
 1   source_id            337727 non-null  int64 
 2   duration             337727 non-null  int64 
 3   embeddings           337727 non-null  object
 4   cosine_similarities  337727 non-null  object
dtypes: int64(3), object(2)
memory usage: 12.9+ MB
